# Header

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D
import scipy as s
%matplotlib inline
#%matplotlib notebook

In [26]:
# Check directory
os.chdir("D:\Raphael\Dropbox\Mcgill\JupyterNotebook\PHYS512")
os.getcwd()

'D:\\Raphael\\Dropbox\\Mcgill\\JupyterNotebook\\PHYS512'

# Functions

In [189]:
# Generate random inital condition
def init_cond(n):
    x=np.random.rand(n,3)
    v=np.random.rand(n,3)*0
    m=np.ones(n)/n
    return x,v,m

In [195]:
# Create density matrix
def rho(x,bin):
    rho = np.zeros(shape=(bin,bin,bin)) #Define the rho matrix
    
    # Now let's go inside each individual 3D bin
    for i in range(bin):
        for j in range(bin):
            for k in range(bin): 
                #Now we check in x for any value which goes into that specific bin (either in x,y or z)
                TF_x0_min = i/bin<=np.transpose(x)[0] 
                TF_x0_max = np.transpose(x)[0]<(i+1)/bin
                TF_x1_min = j/bin<=np.transpose(x)[1] 
                TF_x1_max = np.transpose(x)[1]<(j+1)/bin
                TF_x2_min = k/bin<=np.transpose(x)[2] 
                TF_x2_max = np.transpose(x)[2]<(k+1)/bin
                #Now we can start counting to set the rho matrix
                count = 0
                for x_i in range(len(x)):
                    #Check if this point is indeed in the bin we are looking for.
                    if TF_x0_min[x_i] and TF_x0_max[x_i] and TF_x1_min[x_i] and TF_x1_max[x_i] and TF_x2_min[x_i] and TF_x2_max[x_i]: 
                        count = count+1
                rho[i,j,k]=count
    return rho

In [191]:
#Return a 3D plot of the given x
def plot_graph3D(x, title ="", scale_factor = 1,save=False, name = "fig",n = 0):
    #Create figure
    fig=plt.figure(figsize=(10,10))#Create 3D axes
    ax=Axes3D(fig) 
    ax.scatter(x[0], x[1], x[2], color="royalblue", marker=".",s=.02*scale_factor)
    ax.set_xlabel("x-coordinate",fontsize=14)
    ax.set_ylabel("y-coordinate",fontsize=14)
    ax.set_label("z-coordinate",fontsize=14)
    ax.set_title(title,fontsize=20)
    #ax.legend(loc="upper left",fontsize=14)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.zaxis.set_ticklabels([])
    if save:
        plt.savefig('Pictures_Final_Project/{}{}.png'.format(name,n), dpi=100)
    plt.close(fig)

# Classes

In [141]:
class Particles:
    def __init__(self,x,v,m):
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.f=np.empty(x.shape)
        self.n=self.x.shape[0]
    def get_forces(self):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    r=np.sqrt(drsqr)
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]/(r*drsqr)
    def update(self,dt):
        self.get_forces()
        self.x=self.x+dt*self.v
        self.v=self.v+dt*self.f

# Code

In [192]:
x,v,m = init_cond(2)
rho_x = rho(x,100)

In [ ]:
y,v1,m1 = init_cond(100)
np.sum(rho(y,500))

In [ ]:
n=2
x=np.random.rand(n,3)
v=np.random.rand(n,3)*0
m=np.ones(n)
parts=Particles(x,v,m)
dt=0.1
for i in range(50):
    plot_graph3D(np.transpose(x),scale_factor=10000,save=True,n=i)
    parts.update(dt)

In [136]:
# define a cube to define the softened potential
size = 101
x = np.zeros(shape=(size**3,3))
x_i = 0
for i in range(size):
    for j in range(size):
        for k in range(size):
            x[x_i] = np.array([i-(size-1)/2,j-(size-1)/2,k-(size-1)/2])
            x_i = x_i + 1
G_laplacian = soft_pot(x,1)

In [124]:
rho = x_i*0

phi = s.ifft(s.fft(G_laplacian) * s.fft(rho))

ValueError: Invalid axis (-1) specified.

In [ ]:
# define softened potential
def soft_pot(x,a):
    x2 =x**2
    r = np.zeros(len(x))
    pot = r*0
    for i in range(len(x)): 
        r[i] = np.sqrt(np.sum(x2[i]))
        if r[i]**3<=a:
            pot[i] = r[i]/a
        else:
            pot[i] = 1/r[i]**2
    return pot